In [1]:
import requests as r
import pandas as pd
import numpy as np
import time
import csv
import json
import matplotlib.pyplot as plt
from datetime import datetime,date
import plotly as py
from pprint import pprint
import re
import scipy
import scipy.stats
import plotly.express as px

In [2]:
import plotly.graph_objects as go
import plotly.offline as pyo
pyo.init_notebook_mode(connected=True)

In [3]:
day_from = 1
month_from = 4
year_from = 2020

day_to = 10
month_to = 5
year_to = 2020

#С какой даты
date_from = datetime(year = year_from, month = month_from, day = day_from, hour = 0, minute = 0, second = 0)
#По какую дату
date_to = datetime(year = year_to, month = month_to, day = day_to, hour = 23, minute = 59, second = 59)

#DO NOT CHANGE
token_vk = '23acc95023acc95023acc9504023c092a1223ac23acc9507ef4dc240205bcafea27244d' #vk service token
token_tgstat = '4bb1914294369f7ba8507a33904eb3e7' #tgstat token
#token = '85321298e85028300d359d90f3e9a8def0a98f5b5f26ff14393b7c92204eb5efdc4164ee9a8e5c6265e41' #user (i_am_frodo) token
version = 5.101

groups = [['social_media',"1T3ZCbGNqsWn12J9JuUvITTKneOTdC1kYadc-13aPIaA"],
          ['all_world',"1t801nWYO3tkJA3Mz32cMAKAvmmeNVBxNrACZC7w4C84"],
          ['business',"1HX1plvc3CqfQrloVWTOqRbrujX4CdJUh_-JmxSaTJ8Q"],
          ['creative',"1U3KFmJoIpG_lspueZsh3jOnAXsBJnAq5tmYh675BHs8"],
          ['education',"1mKaSGAvI92mImMTHVOQ2rLSxbuVa3LkSoL38__v-Eoo"],
          ['entertainment',"1TsO6PNjLTZ37haR2CCnb7Iyxprj0IFVqECCImoJUxs0"],
          ['faculty',"1FaliiIbDpnqKqokGrtCpuvBvsepLY48YEyjHR2uYE5c"],
          ['good',"18z_0rN9UV5J6Tp3LBjAo4OWLFJRUocOhVYQiep8kshA"],
          ['sport',"1CmTtB4q1r796h_Wdg4qIWpiMUsqdPmast-ovKrA7o88"]]

In [4]:
def get_members(group_id):
    try_count = 0
    while (try_count<2):
        try:
            response = r.get('https://api.vk.com/method/groups.getById',
                 params={
                     'v' : version ,
                     'access_token' : token_vk,
                     'group_ids' : group_id,
                     'fields' : 'members_count'
            })
            return response.json()['response'][0]['members_count']
        except:
            try_count +=1;
        time.sleep(0.06)

In [5]:
def cleanText(raw_text):
    cleanr=re.compile('<.*?>|(\[.*?\|)|\]')
    cleantext = re.sub(cleanr,'',raw_text)
    return cleantext

In [6]:
def get_reposts_tg(df):
    reposts_list = [];
    for post in  df.iterrows():
        post_id = post[1]['post link']
        res = r.get('https://api.tgstat.ru/posts/stat',
             params={
                 'token' : token_tgstat,
                 'postId' : post_id,
             })
        try:
            response = res.json()['response']
        except:
            print(post_id,' failed on ',res.json()['error'], 'in get_reposts_tg()')
            count_ok = False;
            continue;
        reposts = response['forwardsCount'] + response['mentionsCount']
        reposts_list.append(reposts);
    df['reposts'] = reposts_list;
    return df;

In [7]:
def sort_posts(posts_df):
    posts_df.sort_values(['views'], axis=0, ascending=False, inplace=True)
    posts_df.reset_index(drop=True,inplace=True)
    index_range = len(posts_df.index.values)
    for i in range(index_range):
        new_index=list(posts_df.index.values)
        top_value_index = i;
        current_group = posts_df.iloc[i]['group name']
        count = 0
        for j in range(i+1,index_range):
            if (posts_df.iloc[j]['group name'] == current_group and count < 2):
                del new_index[j]
                new_index.insert(i+1,j)
                i+=1
                count +=1
        posts_df= posts_df.reindex(new_index)
        posts_df.reset_index(drop=True,inplace=True)
    return posts_df

In [8]:
def analyse_tg(group_id):
    
    headers = ["group name","post date","post link", "text", "views","ERR, %"]
    top_viewed_posts = pd.DataFrame(columns=headers)
    all_posts_period = pd.DataFrame(columns=headers)
    
    def skip_group(value):
        posts_list.append(value)
        views_list.append(value)
        members_list.append(value)
        err_list.append(value)
        
        
    # начинаем проходить по группам
    for group in  df.iterrows():
        group_id = group[1]['telegram']
        group_name = group[1]['group_name']
        views_counter = 0;
        post_counter = 0;
        posts_in_group = [];
        
        # если нет группы в тг, то всё по нулям
        if (pd.isna(group_id)):
            skip_group(''); 
            continue;
        
        offset = 0;
        count_ok = True;
        members = 0;
        while (count_ok):
            res = r.get('https://api.tgstat.ru/channels/posts',
                 params={
                     'token' : token_tgstat,
                     'channelId' : group_id,
                     'offset' : offset,
                     'limit': 50,
                     'startTime': int(date_from.timestamp()),
                     'endTime': int(date_to.timestamp()),
                     'hideDeleted': 1,
                     'hideForwards':1,
                     'extended': 1
                 })
            try:
                response = res.json()['response']
            except:
                print(group_id,' failed on ',res.json()['error'] )
                count_ok = False;
                continue;
            members = response['channel']['participants_count']
            if (response['count']==0): #если в выгрузке пусто, переходим к следующей группе
                count_ok = False;
                continue;
           
            #проверяем что не вылезли за размеры выгрузки
            if (offset + response['count'] != response['total_count']):
                offset += response['count'];
            else:
                count_ok = False;
            
            post_counter = response['total_count']
            #считаем посты и выгружаем их 
            all_posts = response['items']
            for post in all_posts:
                post_info = []
                post_ERR = post['views']/ members * 100
                post_date = datetime.fromtimestamp(post['date'])
                post_text = cleanText(post['text'])
                group_info = response['channel']['title'] +' ('+str(members)+')';
                post_info.append((group_info,post_date,post['link'],post_text,post['views'],post_ERR))
                views_counter += post['views']
                posts_in_group.extend(post_info);
                
        members_list.append(members)
        if (post_counter):
            posts_list.append(post_counter)
            views_list.append(round(views_counter/post_counter))
            err_list.append(round(round(views_counter/post_counter)/members * 100,2))
        else:
            posts_list.append(0)
            views_list.append(0)
            err_list.append(0)
        posts_data = pd.DataFrame(posts_in_group, columns=headers)
        posts_data.sort_values(["views"], axis=0, ascending=False, inplace=True)
        all_posts_period = pd.concat([all_posts_period,posts_data],sort=False)
        
        i=0;
        nextpost = 0;
        while i<3:
            try:
                if (nextpost and posts_data.iloc[nextpost,1] == posts_data.iloc[nextpost-1,1]):
                    nextpost+=1;
                    continue;
            except IndexError:
                break
            try:
                top_viewed_posts = top_viewed_posts.append(posts_data.iloc[[nextpost]])
                nextpost+=1
                i+=1
            except:
                nextpost+=1                
                continue
                
#     export_name_all = 'results/'+group_list+' '+str(day_from)+'.'+str(month_from)+"-"+str(day_to)+'.'+str(month_to)+' TG all posts.xlsx'
#     all_posts_period.to_excel(export_name_all, encoding='utf-8',engine='xlsxwriter')
    
    df=df.join(pd.DataFrame({
        'tg_members' :members_list,
        'tg_posts' : posts_list,
        'tg_avg_views' : views_list,
        'tg_avg_ERR' : err_list
    }, index=df.index));

    return df,all_posts_period

In [22]:
def analyse_vk(group_list,df,save):
    likes_list,posts_list,members_list, views_list,err_list = ([] for i in range(5));
    
    headers = ['group name','post date','post link', 'text', 'views','ERR, %','likes','reposts','comments']
    top_viewed_posts = pd.DataFrame(columns=headers)
    all_posts_period = pd.DataFrame(columns=headers)
    
    def skip_group(value):
        posts_list.append(value)
        likes_list.append(value)
        views_list.append(value)
        err_list.append(value)
        
    # начинаем проходить по группам
    for group in  df.iterrows():
        group_id = group[1]['vk']
        group_name = group[1]['group_name']
        like_counter = 0;
        post_counter = 0;
        views_counter = 0;
        posts_in_group = [];
        # если нет группы в вк, то всё по нулям
        if (pd.isna(group_id)):
            skip_group('')
            members_list.append('')
            continue;
        
        offset = 0;
        members = get_members(group_id)
        
        date_ok = True;
        last_try = 0;
        #Выгружаем посты на стенке, пока не выйдем за "левую" дату
        
        while (date_ok or last_try <=1):
            res = r.get('https://api.vk.com/method/wall.get',
                 params={
                     'v' : version,
                     'access_token' : token_vk,
                     'domain' : group_id,
                     'offset' : offset,
                     'count': '100'
                 })
            try:
                response = res.json()['response']
            except:
                date_ok = False;
                last_try = 2;
                continue;
                
            if (response['count']==0): #если в выгрузке пусто, переходим к следующей группе
                date_ok = False;
                last_try = 2;
                continue;
            
            #считаем посты удовлетворяющие условию по датам  
            all_posts = response['items']
            if all(datetime.fromtimestamp(post['date']) < date_from 
                  for post in all_posts):
                date_ok = False
                last_try += 1;
            else:
                for post in all_posts:
                    post_info = []
                    post_date = datetime.fromtimestamp(post['date'])
                    if (post_date > date_from and post_date < date_to):
                        post_counter += 1;
                        like_counter += post['likes']['count']
                        group_info = group_name+' ('+str(members)+')';
                        post_link='https://vk.com/wall'+str(post['owner_id'])+'_'+str(post['id'])
                        post_text = cleanText(post['text'])
                        try:
                            views = post['views']['count']
                            views_counter += views
                            post_ERR = round(post['views']['count']/ members * 100,2)
                            post_info.append((group_info,post_date,post_link,post_text,
                                          views,post_ERR,post['likes']['count'],post['reposts']['count'],post['comments']['count']));
                        except:
                            post_info.append((group_info,post_date,post_link,cleanText(post['text']),
                                          0,0,post['likes']['count'],post['reposts']['count'],post['comments']['count']));
                        posts_in_group.extend(post_info)
                offset += len(all_posts)
            time.sleep(0.06)
        
        members_list.append(members)
        likes_list.append(like_counter)
        if (post_counter):
            posts_list.append(post_counter)
            views_list.append(round(views_counter/post_counter))
            err_list.append(round(round(views_counter/post_counter)/members * 100,2))
        else:
            posts_list.append(0)
            views_list.append(0)
            err_list.append(0)
            
        posts_data = pd.DataFrame(posts_in_group, columns=headers)
        all_posts_period = pd.concat([all_posts_period,posts_data],sort=False)
        posts_data.sort_values(['views'], axis=0, ascending=False, inplace=True)
        for i in range(3):
            try:
                top_viewed_posts = top_viewed_posts.append(posts_data.iloc[[i]])
            except:
                continue
                
#     export_name_all = 'results/'+group_list+' '+str(day_from)+'.'+str(month_from)+"-"+str(day_to)+'.'+str(month_to)+' VK all posts.xlsx'
#     all_posts_period.to_excel(export_name_all, encoding='utf-8',engine='xlsxwriter')
    
    df=df.join(pd.DataFrame(
    {
        'vk members' : members_list,
        'vk posts' : posts_list,
        'vk likes' : likes_list,
        'vk avg views' : views_list,
        'vk avg err' : err_list
    }, index=df.index )) 
    new_order = [0,1,4,5,6,7,8,2,3]
    df = df[df.columns[new_order]]

    if (group_list == "social_media" and save):
        top_viewed_posts = sort_posts(top_viewed_posts)
        export_name = 'results/'+group_list+' '+str(day_from)+'.'+str(month_from)+"-"+str(day_to)+'.'+str(month_to)+' vk posts.xlsx'
        top_viewed_posts.to_excel(export_name, encoding='utf-8',engine='xlsxwriter')
    
    return df,all_posts_period

In [23]:
def analyse(gr,vk=False,tg=False,graph=False,save=False):
    if (gr=='all'):
        for group in groups:
            source_file = group[0] +'.csv'
            df = pd.read_csv(source_file)
            if (vk):
                df,vk_posts = analyse_vk(group[0],df,save)
            if (tg and group[0]=='social_media'):
                df,tg_posts = analyse_tg(group[0],df,save)
            print (group[0]+ ' is done')
    else:
        source_file = gr +'.csv'
        df = pd.read_csv(source_file)
        if (vk):
            df,vk_posts = analyse_vk(gr, df,save)
        if (tg and gr=='social_media'):
            df,tg_posts = analyse_tg(gr,df,save)
        print (gr+ ' is done')
    return vk_posts #,tg_posts

In [25]:
vk_df = analyse('social_media',vk=True, tg=False, save = False)
vk_df.reset_index(inplace = True)
del vk_df['index']
vk_df.views[0]
vk_df.views = vk_df.views.apply(pd.to_numeric)
vk_df.head()

social_media is done


,group name,post date,post link,text,views,"ERR, %",likes,reposts,comments
0,The Vyshka (58595),2020-05-02 21:00:09,https://vk.com/wall-66036248_36519,У нас новый выпуск подкаста «Вырастешь – пойме...,10597,18.09,103,12,1
1,The Vyshka (58595),2020-05-10 21:41:57,https://vk.com/wall-66036248_36592,Выпускаем второй материал совместно с «Диссерн...,6053,10.33,54,1,2
2,The Vyshka (58595),2020-05-10 16:00:12,https://vk.com/wall-66036248_36590,Когда все сериалы из списка «на потом» просмот...,7469,12.75,59,3,0
3,The Vyshka (58595),2020-05-10 12:00:13,https://vk.com/wall-66036248_36585,"Пока спортивные залы закрыты, единственный вар...",7221,12.32,56,7,2
4,The Vyshka (58595),2020-05-09 21:00:11,https://vk.com/wall-66036248_36584,"«Это не столько замкнутое сообщество, сколько ...",6441,10.99,64,3,0


In [29]:
vk_df['group name'].value_counts()

Студенческий городок Дубки (11570)         439
DOXA (10987)                               160
The Vyshka (58595)                         119
HSR(High School Radio) (8039)               79
HSP(High School Press) (7933)               38
Studlife HSE (16902)                        22
THE WALL Magazine (8807)                    10
МЕДИАЦЕНТР Высшей школы экономики (668)      9
Студенческий Совет НИУ ВШЭ (6974)            9
ТелеВышка (5291)                             7
Name: group name, dtype: int64

In [33]:
Doxa = vk_df[vk_df['group name'] == 'DOXA (10987)']
mean_posts_week = int(Doxa.groupby(Doxa['post date'].dt.to_period('w')).count()['views'].mean())
mean_views = int(Doxa.groupby(Doxa['post date'].dt.to_period('w')).mean()['views'].mean())
sum_views_week = int(Doxa.groupby(Doxa['post date'].dt.to_period('w')).sum()['views'].mean())
std_week = int(Doxa.groupby(Doxa['post date'].dt.to_period('w')).std()['views'].mean())
print('mean posts:',mean_posts_week, '\nmean views:', mean_views,
      '\nsum views per week:', sum_views_week,'\nstd per week:', std_week)
#scipy.stats.poisson.pmf(k, mu, loc=0.35) 
#k - проверяемые на аномальность просмотры, mu - среднее количество просмотров в проверяемый период
probs = []
for i in range(100,5000):
    probs.append(scipy.stats.poisson.pmf(i,mean_views))

fig = go.Figure()
fig.add_trace(go.Scatter(x=list(range(100,5000)), y=probs, name = 'probs'))
fig.show()

mean posts: 26 
mean views: 1821 
sum views per week: 48647 
std per week: 1087


In [36]:
def is_anomaly_poisson(views):
    return(scipy.stats.poisson.pmf(views,mean_views) == 0 and views > mean_views)
            
for index, row in Doxa.iterrows():
    if is_anomaly_poisson(row.views):
        print('anomaly found:',row.views,row['post link'])

anomaly found: 12170 https://vk.com/wall-128503206_6768
anomaly found: 3785 https://vk.com/wall-128503206_6745
anomaly found: 6829 https://vk.com/wall-128503206_6675
anomaly found: 7621 https://vk.com/wall-128503206_6596
anomaly found: 3999 https://vk.com/wall-128503206_6516
anomaly found: 6196 https://vk.com/wall-128503206_6403
anomaly found: 4707 https://vk.com/wall-128503206_6367


In [38]:
def three_sigma_anomaly(views):
    ano_cut_off = 3*std_week
    upper_cut = mean_views + ano_cut_off
    return(views>upper_cut)

for index, row in Doxa.iterrows():
    if three_sigma_anomaly(row.views):
        print('anomaly found:',row.views,row['post link'])

anomaly found: 12170 https://vk.com/wall-128503206_6768
anomaly found: 6829 https://vk.com/wall-128503206_6675
anomaly found: 7621 https://vk.com/wall-128503206_6596
anomaly found: 6196 https://vk.com/wall-128503206_6403


In [43]:
def to_hundr(num):
#     num = int(num/100)*100
    if num <= 10000:
        return int(num/1000)
    else: return 10
    
arr = np.array(vk_df.views.apply(to_hundr))
poi = scipy.stats.poisson.rvs(0.4, size=len(arr))
ber = scipy.stats.bernoulli.rvs(0.5, size=len(arr))
views, view_count = np.unique(arr,return_counts = True)
poi_val, poi_count = np.unique(poi,return_counts = True)
fig = go.Figure()
fig.add_trace(go.Scatter(x=views, y=view_count, name = 'vk'))
fig.add_trace(go.Scatter(x=poi_val, y=poi_count, name = 'poisson'))
fig.update_layout(title = 'распределение просмотров вк')
fig.show()

In [49]:
def to_hundr(num):
#     num = int(num/100)*100
    if num <= 10000:
        return round(num/1000)
    else: return 10
    
# arr = np.array(vk_df.views)
# poi = np.random.poisson(0.4,size= len(arr))
# views, view_count = np.unique(arr,return_counts = True)
# poi_val, poi_count = np.unique(poi,return_counts = True)
# fig = go.Figure()
# fig.add_trace(go.Scatter(x=views, y=view_count, name = 'vk'))
# fig.add_trace(go.Scatter(x=poi_val, y=poi_count, name = 'poisson'))
# fig.update_layout(title = 'распределение просмотров вк')
# fig.show()
fig = px.histogram(Doxa, x="views",nbins = 80, title = 'распределение просмотров отдельно взятого сообщества')
fig.show()

In [ ]:
def get_reposts_list(post_id):
    res=r.get('https://api.tgstat.ru/posts/stat',
         params={
         'token' : token_tgstat,
         'postId' : post_id   
         })
    reposts = [];
    forwards = res.json()['response']['forwards']
    mentions = res.json()['response']['mentions']
    for f in forwards:
        reposts.append(f['postLink'])
    for m in mentions:
        reposts.append(m['postLink'])
    
    print('Reposts for ',post_id,':\n',str(reposts))    

In [ ]:
get_reposts_list('t.me/doxajournal/3455')